In [2]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

In [57]:
import os
current_path = os.path.abspath(os.getcwd())
datasets_directory = os.path.join(current_path, 'datasets')

if not os.path.exists(datasets_directory):
    os.makedirs(datasets_directory)

In [27]:
connection = create_connection()

# Get data

In [28]:
data = articles.get_articles(connection)

In [29]:
pd.set_option('max_colwidth', 100)
df = pd.DataFrame(data, columns = ['publication_date',
                                   'author',
                                   'title',
                                   'url',
                                   'koronawirus_in_text',
                                   'koronawirus_in_title',
                                   'question_mark',
                                   'exclamation_mark',
                                   'all_words'])

In [31]:
df.sample(frac = 0.001)

,publication_date,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
8104,2020-07-13 13:38:00,Mariusz Mucha,"Wąwolnica: Oglądali dom na sprzedaż, gdy stało się TO. Coś strasznego!",https://lublin.se.pl/wawolnica-ogladali-dom-na-sprzedaz-kiedy-stalo-sie-to-aa-gQP4-yZJi-axp9.html,0.0,0.0,0.0,1.0,222.0
1732,NaT,None,None,https://www.se.pl/lublin/ulicami-lublina-przejdzie-ii-marsz-rownosci-bedzie-powtorka-z-bialegost...,NaN,NaN,NaN,NaN,NaN
4918,NaT,None,None,https://www.se.pl/lublin/wsadzili-suczke-do-torby-i-wyrzucili-przy-drodze-ruszyla-zbiorka-na-lec...,NaN,NaN,NaN,NaN,NaN
7888,2020-08-30 17:32:00,Karolina Januszek,Gang Fajniaków w Biedronce. Jak odebrać naklejki w kasie samoobsługowej?,https://lublin.se.pl/gang-fajniakow-w-biedronce-jak-odebrac-naklejki-w-kasie-samoobslugowej-ak-Z...,0.0,0.0,2.0,0.0,350.0
4513,NaT,None,None,https://www.se.pl/lublin/napad-na-bank-w-lublinie-policja-szuka-sprawcy-aa-7JuX-AKXg-4RHS.html,NaN,NaN,NaN,NaN,NaN
2383,NaT,None,None,https://www.se.pl/lublin/takie-zachowanie-dziecka-powinno-niepokoic-czym-jest-autyzm-audio-aa-ii...,NaN,NaN,NaN,NaN,NaN
8001,2020-08-04 08:18:00,Marek Targoński,Koronawirus: Ponad 1000 przypadków w Lubelskiem. 11 aktywnych ognisk COVID-19 [MAPA],https://lublin.se.pl/koronawirus-ponad-1000-przypadkow-w-lubelskiem-11-aktywnych-ognisk-covid-19...,18.0,1.0,2.0,1.0,166.0
8324,2020-05-06 10:44:00,mt,Koronawirus na Lubelszczyźnie. Są kolejne przypadki zakażeń. Dramatyczne dane z Polski,https://www.se.pl/lublin/koronawirus-na-lubelszczyznie-sa-kolejne-przypadki-zakazen-dramatyczne-...,24.0,1.0,2.0,0.0,218.0
3248,NaT,None,None,https://www.se.pl/lublin/zamosc-100-ksiazek-na-100-lat-niepodleglosci-wynik-zaskoczyl-audio-aa-L...,NaN,NaN,NaN,NaN,NaN


# Clean data

### initial data check, organize and clearing

In [32]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8587 entries, 0 to 8586
Data columns (total 9 columns):
publication_date        1651 non-null datetime64[ns]
author                  1651 non-null object
title                   1651 non-null object
url                     8587 non-null object
koronawirus_in_text     1651 non-null float64
koronawirus_in_title    1651 non-null float64
question_mark           1651 non-null float64
exclamation_mark        1651 non-null float64
all_words               1651 non-null float64
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 2.6 MB


In [33]:
df.nunique()

publication_date        1641
author                   126
title                   1648
url                     8587
koronawirus_in_text       35
koronawirus_in_title       2
question_mark             18
exclamation_mark          15
all_words                383
dtype: int64

In [34]:
df.dropna(inplace=True)
df.shape

(1651, 9)

In [35]:
df['author'] = df['author'].astype('category')
df['koronawirus_in_text'] = df['koronawirus_in_text'].astype('int')
df['koronawirus_in_title'] = df['koronawirus_in_title'].astype('int')
df['question_mark'] = df['question_mark'].astype('int')
df['exclamation_mark'] = df['exclamation_mark'].astype('int')
df['all_words'] = df['all_words'].astype('int')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 9 columns):
publication_date        1651 non-null datetime64[ns]
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(2)
memory usage: 765.5 KB


In [36]:
df.count()

publication_date        1651
author                  1651
title                   1651
url                     1651
koronawirus_in_text     1651
koronawirus_in_title    1651
question_mark           1651
exclamation_mark        1651
all_words               1651
dtype: int64

### column: publication_day

In [37]:
df.insert(loc=1, column='publication_day', value=df['publication_date'].dt.strftime('%Y-%m-%d'))
df.sample(frac = 0.002)

,publication_date,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
834,2020-02-27 08:39:00,2020-02-27,mt,"Chrząchów: Pożar fermy, w środku pięć tysięcy indyków. NOCNY KOSZMAR zwierząt [ZDJĘCIA]",https://www.se.pl/lublin/chrzachow-piec-tysiecy-indykow-splonelo-w-pozarze-nocny-koszmar-zwierza...,0,0,0,2,133
821,2020-03-05 19:43:00,2020-03-05,mt,"Łęczna: Trzymał w mieszkaniu... MŁODĄ SARNĘ! Tłumaczył, że ma ją od urodzenia",https://www.se.pl/lublin/leczna-trzymal-w-mieszkaniu-mloda-sarne-tlumaczyl-ze-ma-ja-od-urodzenia...,0,0,0,1,185
336,2020-07-10 11:32:00,2020-07-10,OM,"Pozbądź się starego płaszcza, palta, czy futra, a przy okazji pomóż zorganizować Noc Kultury",https://lublin.se.pl/pozbadz-sie-starego-plaszcza-palta-czy-futra-a-przy-okazji-pomoz-zorganizow...,0,0,1,0,214


In [38]:
df.publication_day.value_counts().head()

2020-06-29    28
2020-07-08    26
2020-07-01    25
2020-07-03    22
2020-08-25    22
Name: publication_day, dtype: int64

In [39]:
df['publication_day'].nunique()

233

In [40]:
#df['publication_day'] = df['publication_day'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 10 columns):
publication_date        1651 non-null datetime64[ns]
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(3)
memory usage: 873.5 KB


### column: publication_month

In [41]:
df.insert(loc=1, column='publication_month', value=df['publication_date'].dt.strftime('%Y-%m'))
df.sample(frac = 0.001)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
8322,2020-05-06 20:46:00,2020-05,2020-05-06,mt,"Lublin: Biuro paszportowe w nowym miejscu. Gdzie po paszport w Lublinie? [ADRES, INFORMATOR]",https://www.se.pl/lublin/lublin-biuro-paszportowe-w-nowym-miejscu-gdzie-po-paszport-w-lublinie-a...,4,0,3,1,373
275,2020-07-19 10:42:00,2020-07,2020-07-19,Mariusz Mucha,Wiózł tytoń zamiast cegieł. Zatrzymali przemytnika z Węgier,https://lublin.se.pl/wiozl-tyton-zamiast-cegiel-zatrzymali-przemytnika-z-wegier-aa-riaD-qMVy-JnK...,0,0,0,0,96


In [42]:
df.publication_month.value_counts().head(8)

2020-07    439
2020-08    301
2020-06    214
2020-01    154
2020-05    153
2020-03    147
2020-04    138
2020-02    105
Name: publication_month, dtype: int64

In [43]:
df['publication_month'].nunique()

8

In [44]:
#df['publication_month'] = df['publication_month'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 11 columns):
publication_date        1651 non-null datetime64[ns]
publication_month       1651 non-null object
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(4)
memory usage: 976.7 KB


### column: author

In [45]:
df['author'].value_counts()[:60]

mt                                 335
Marek Targoński                    183
Mucha                              147
Karolina Januszek                  131
Mariusz Mucha                      111
MTA                                 80
Mucha, mta                          55
AP                                  36
KM                                  33
OM                                  32
Bartłomiej Ważny                    23
Monika Kowalewicz                   22
AC                                  21
Michał Michalak                     19
EIB                                 19
Emilia Białecka                     16
MK                                  16
Mateusz Kasiak                      16
Agnieszka Niećko                    15
Jacek Werner                        13
gk                                  13
Olka Mazur                          13
Artykuł sponsorowany                13
Sylwia Sitka-Czerniak               13
Grzegorz Kluczyński                 13
maal                     

In [46]:
df['author'] = df['author'].str.lower()
regex_pattern = re.compile(r'.*mt.*', re.I)
df[df['author'].str.contains(regex_pattern)].count()

publication_date        514
publication_month       514
publication_day         514
author                  514
title                   514
url                     514
koronawirus_in_text     514
koronawirus_in_title    514
question_mark           514
exclamation_mark        514
all_words               514
dtype: int64

In [49]:
df['author'] = df['author'].replace(['mt', 'mt; wideo: Tygodnik Zamojski', 'gał'],'marek targoński')
# df['author'] = df['author'].replace(['Mucha', 'mucha'],'Mariusz Mucha')
# df['author'] = df['author'].replace(['ŁT'],'Łukasz Trybulski')
# df['author'] = df['author'].replace(['Mateusz Kasiak (Radio Eska)'],'Mateusz Kasiak')
# df['author'] = df['author'].replace(['KaJa'],'Karolina Januszek')
# df['author'] = df['author'].replace(['RS'],'Rafał Strzelec')
# df['author'] = df['author'].replace(['gk'],'Grzegorz Kluczyński')
# df['author'] = df['author'].replace([''],'Nieznany')

# regex_pattern = re.compile(r'(.*,.*|.*/.*|Redakcja ESKA INFO|Nieznany|Akcja partnerska)', re.I)
# df['author'] = df['author'].replace(regex_pattern,'Więcej autorów lub nieznany')

regex_pattern = re.compile(r'.*mt.*', re.I)
df['author'] = df['author'].replace(regex_pattern,'marek targoński')

df['author'].value_counts()[:5]

marek targoński      699
mucha                148
karolina januszek    131
mariusz mucha        111
ap                    36
Name: author, dtype: int64

In [50]:
df['author'] = df['author'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 11 columns):
publication_date        1651 non-null datetime64[ns]
publication_month       1651 non-null object
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(4)
memory usage: 971.3 KB


### column: koronawiorus_text

In [51]:
# df.loc[[919, 1979, 1965], 'koronawirus_in_text'] = 0

### column: koronawiorus_anywhere_sum

In [52]:
df['koronawirus_anywhere'] = 0
df['koronawirus_anywhere'] = df['koronawirus_in_text'] + df['koronawirus_in_title']
# df.loc[df.koronawirus_anywhere > 0, 'koronawirus_anywhere'] = 1
df.head(1)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere
0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-...,2,0,1,0,263,2


### column: koronawiorus_anywhere_count

In [53]:
df['koronawirus_anywhere_count'] = 0
df['koronawirus_anywhere_count'] = (df['koronawirus_in_text'] + df['koronawirus_in_title']) / (df['koronawirus_in_text'] + df['koronawirus_in_title'])
df['koronawirus_anywhere_count'] = df['koronawirus_anywhere_count'].fillna(0).astype('int')
df.head(3)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-...,2,0,1,0,263,2,1
1,2020-08-28 20:45:00,2020-08,2020-08-28,mariusz mucha,Kiełbasiany ZŁODZIEJ wpadł w Lublinie. Rzucił się Z TULIPANEM na ochroniarza!,https://lublin.se.pl/kielbasiany-zlodziej-wpadl-w-lublinie-aa-Phpp-QAn1-bHnw.html,0,0,0,0,243,0,0
2,2020-08-28 13:43:00,2020-08,2020-08-28,eib,Jaka będzie pogoda w ostatni weekend wakacji? Szykujcie się na najgorsze!,https://lublin.se.pl/jaka-bedzie-pogoda-w-ostatni-weekend-wakacji-szykujcie-sie-na-najgorsze-ak-...,0,0,1,1,171,0,0


### Export data

In [58]:
df.to_csv('datasets/1_clean_data.csv')